# Latent Semantic Analysis: From Autoencoders to Embeddings

## Part I: The Autoencoder

In this notebook, we explore using an autoencoder for Latent Semantic Analysis (LSA). The goal of LSA is to embed words from running text into a latent space in a way that captures the structure and relatedness of these words. An autoencoder is a neural network which is designed to compress arbitrary high-dimensional data. To accomplish this, we start with a layer the size of the high-dimensional data, and iteratively feed this data into layers of lower and lower dimension until we reach a low-dimensional layer. We then feed data from this low dimensional layer into layers of higher and higher dimension, increasing the dimensionality back to the original dimensionality. Then, we train the network on a loss function which seeks to minimize the difference between inputs and outputs. The network then learns to optimally compress the high dimensional data in such a way that it can be restored with maximum accuracy. 

<img src="files/autoencoder-architecture.png">

The autoencoder architecture is summarized above. Here are the specifications of the autoencoder we will use:

1. We use just three layers for our autoencoder: an input layer, a hidden layer representing the latent space, and an output layer.
2. We will not introduce any nonlinearities on the first two layers. On the third layer, we will introduce a sigmoid function.
3. We will train with cross-entropy loss to match the sigmoid of the last layer.

We have yet to see how we will turn running text into data which our autoencoder can use. The first and most obvious step is to perform a one-hot encoding of the words we see in our text corpus. We may be tempted to naively try and compress the one-hot encodings by feeding them into the autoencoder one-by-one and backpropogating any difference between outputs and inputs. This will not work very well, because autoencoders can only compress data well when the high-dimensional data lies in some (perhaps nonlinear) low-dimensional subspace. Here, we have a data point corresponding to each unit vector (because of one-hot encoding), so the autoencoder is essentially trying to compress the entire space. The result is that the space cannot be compressed any more efficiently than a random linear embedding. This is not surprising in light of the fact that we haven't incorporated any information about the words and how they are related (other than that they are all distinct).

To solve this issue, we will instead train our autoencoder to output "context words" for a given input word. A "context word" $w'$ for a given input word $w$ is simply a word which appears near $w$ in the corpus. Our data matrix will simply be the identity, since we intend to train the autoencoder on one-hot encoding inputs. The target matrix has rows indexed by input and columns indexed by output, so the $(i, j)$th component of $y$ is 1 if the word $w'$ whose one-hot index is $j$ appears in within $L$ words of some appearance of $w$, the word whose one-hot index is $i$. Here $L$ is a hyperparameter, which we will refer to as the window size. Notice that the context words for a word $w$ are the words which appear within $2L$ of this word. When we construct $y$, we will do so by starting with all zeroes, indexing through the corpus, and for each center word $w$ and context word $w'$ at most $L$ indices away, we set the corresponding entry of $y$ to 1.

This form of LSA, called "Continuous Bag of Words" (CBOW) LSA, could also be see as the following classification problem: Given a pair $(w', w)$, classify $w'$ as either a context word for $w$ or not a context word for $w$. The output of the autoencoder in index $w'$ when run on the one-hot encoding of $w$ would then be the classification "score" of $(w', w)$: an output of 1 would imply that $w'$ always appears within $L$ of $w$, so $w'$ is certainly a context word, and an ouptut of 0 would imply that $w'$ never appears within $L$ of $w$, so $w'$ is certainly not a context word. 

Later, we will see an embedding-focused approach to CBOW LSA that is rooted in this classification problem, but instead randomly samples positive and negative examples $(w', w)$ on which to train. We will even prove that if we sample every possible example in this alternative model, our results will be identical to those achieved via the autoencoder. Moreover, this approach may be faster and yield better results when we reduce the number of negative examples to be balanced with the number of positive examples. Reducing the number of examples saves training time, and decreasing class imbalance typically improves performance. But first, let's implement the autoencoder and see it in action on a corpus of Wikipedia articles.

In [ ]:
import numpy as np
from random import sample
import sys
import re
import nltk
import tqdm
from tqdm import tqdm
import pickle
from gensim.corpora import WikiCorpus
from nltk.corpus import reuters
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
from gensim.models import LsiModel

In [1]:
class Embedding:
    """A class representing an autoencoder for LSA Embedding

    :param latent_dim: The dimension of the latent space in which to embed words.
    :param corpus: A list containing all the words we wish to embed.
    :param window: The length of the context word window.
    :param load_weights: Whether or not to load a pretrained model we have saved
                            from a previous training session. If a pretrained
                            model is loaded then the fit function is disabled
                            since our model will already be fit to the data.
    :param add_positives: **DO NOT USE** Whether or not to go through the corpus
                            and add positive examples; only set false for the
                            purpose of sanity checking Part (A). 
    :param add_positives: **DO NOT USE** Whether or not to go through the corpus
                            and add positive examples; only set false for the
                            purpose of sanity checking Part (B). 
    """

    def __init__(self, latent_dim, corpus, window=3, load_weights=False, 
                 add_positives=True, normalize_y=True):

        # Load weights if necessary
        if load_weights:
            self.load()
            self.has_data = False
            return
        self.has_data = True

        # Define activation
        self.sigmoid = lambda x: np.divide(1, np.add(1, np.exp(-x)))

        # Separate unique words and find one-hot encoding indices
        self.words = set(corpus)
        self.idx = {word: i for i, word in enumerate(self.words)}
        print(f"found words {len(self.words)} words in a corpus of length {len(corpus)}")

        # Initialize data to all negative examples
        self.X = np.eye(len(self.words))
        ################## BEGIN PART (A) ##################
        # Initialize self.y to contain all negative examples

        ################### END PART (A) ###################

        # Add in all positive examples
        if add_positives:
            for i in range(len(corpus)):
                # Get the one-hot encoding indices of all words in window
                # Remove one-hot encoding index of center word, store separately
                start = max(0, i - window)
                end = min(len(corpus), i + window)
                window_indices = [self.idx[x] for x in corpus[start:end]]
                center_index = window_indices.pop(i - start)
                ################## BEGIN PART (B) ##################
                # Add 1 to self. y in relevant position for each positive example 
                # (w', w), where w is the center and w' in window
                # REMEMBER: first index of y corresponds to w', second index to w

                ################### END PART (B) ###################
        
        if normalize_y:
            ################## BEGIN PART (C) ##################
            # Normalize target so that y[w, w'] is the probability
            # that a randomly chosen context word of w is w'
            # (the sum of y[w, w'] over w' should be 1 for all w)

            ################### END PART (C) ###################

        # Randomly initialize embeddings
        self.weights1 = np.random.rand(latent_dim, len(self.words))
        self.weights2 = np.random.rand(len(self.words), latent_dim)

    def fit(self, lr, epochs):
        if not self.has_data: #Don't train if this is a pre-trained model!
            return
        for i in range(epochs):
            data_bar = tqdm(range(len(self.words)), position=0, leave=True)
            data_bar.set_description(f"Processing epoch {i+1} out of {epochs}")
            for j in data_bar:
                hidden = self.weights1 @ self.X[:, j]
                output = self.sigmoid(self.weights2 @ hidden)
                ################## BEGIN PART (D) ##################
                # Implement Backpropagation

                ################### END PART (D) ###################
        return self

    def _get_one_hot(self, word):
        one_hot = np.zeros(len(self.words))
        ################## BEGIN PART (E) ##################
        # Set relevant index of one_hot to 1
        # Hint: See __init__ code for one-hot indices

        ################### END PART (E) ###################
        return one_hot

    # The reasons behind this naming convention will be apparant shortly
    def word2vec(self, word):
        ################## BEGIN PART (F) ##################
        # Return the embedding (hint: use the first layer)

        ################### END PART (F) ###################
        
    def cos_similarity(self, word1, word2):
        embedding1 = self.word2vec(word1)
        embedding2 = self.word2vec(word2)
        ################## BEGIN PART (G) ##################
        # Return the cosine of the angle between the embeddings

        ################### END PART (G) ###################

    def most_similar(self, word):
        ################## BEGIN PART (H) ##################
        # Return a list of the form [(word1, cos), (word2, cos), ...]
        # where word1, word2, etc are other words and cos1, cos2, etc
        # are the cos similarities with the input word
        # Sort the list in order of decreasing similarity (so the most
        # similar words are at the front)

        ################### END PART (H) ###################

    def save(self):
        kwargs = {'protocol': pickle.HIGHEST_PROTOCOL}
        np.save("weights1", self.weights1)
        np.save("weights2", self.weights2)
        with open('word_indices.pickle', 'wb') as handle:
            pickle.dump(self.idx, handle, **kwargs)

    def load(self):
        self.weights1 = np.load("weights1")
        self.weights2 = np.load("weights1")
        with open('word_indices.pickle', 'rb') as handle:
            self.idx = pickle.load(handle)

Once you've completed the code in the cell above, use the following sanity checks to confirm the correctness of your solutions. Note that the sanity check for Part (B) depends on Part (A), and the sanity check for Part (C) depends on Parts (A) and (B). Other than that, these sanity checks operate independently, so the sanity check for, as an example, Part (E), will reflect the correctness of your solution for Part (E) **regardless** of whether your solution to, say, Part (A) is correct.

In [ ]:
# PART (A) SANITY CHECK
corpus = ['a', 'bunch', 'of', 'words']
embed = Embedding(2, corpus, add_positives=False, normalize_y=False)
if (embed.y != 0).any():
    print("Embedding's y matrix not initialized to zero!")
else:
    print("Sanity Check Passed!")

In [ ]:
# PART (B) SANITY CHECK
words = ['a', 'bunch', 'of', 'words', 'in', 'a', 'bag', 'a']
embed = Embedding(2, words, window=1, normalize_y=False)
target = np.array([[0, 0, 1, 0, 0, 0], 
                   [0, 0, 0, 0, 0, 1], 
                   [0, 0, 0, 1, 0, 0], 
                   [0, 1, 0, 0, 1, 0],
                   [0, 0, 0, 1, 0, 0],
                   [1, 0, 0, 0, 0, 0]])
if (np.abs(embed.y - target) >= 1e-10).any():
    print(embed.y)
    print("Embedding's y matrix not correctly initialized!")
else:
    print("Sanity Check Passed!")

In [ ]:
# PART (C) SANITY CHECK
words = ['a', 'bunch', 'of', 'words', 'in', 'a', 'bag', 'a']
embed = Embedding(2, words, window=1)
target = np.array([[0, 0, 1, 0, 0, 0], 
                   [0, 0, 0, 0, 0, 1], 
                   [0, 0, 0, 1/2, 0, 0], 
                   [0, 1, 0, 0, 1, 0],
                   [0, 0, 0, 1/2, 0, 0],
                   [1, 0, 0, 0, 0, 0]])
if (np.abs(embed.y - target) >= 1e-10).any():
    print("Embedding's y matrix not correctly initialized!")
else:
    print("Sanity Check Passed!")

In [ ]:
# PART (E) SANITY CHECK
words = ['a', 'bunch', 'of', 'words', 'in', 'a', 'bag']
embed = Embedding(2, words, window=1)
condition1 = (np.abs(embed._get_one_hot('bag') - np.array([0, 0, 0, 0, 1, 0])) >= 1e-10).any()
condition2 = (np.abs(embed._get_one_hot('a') - np.array([0, 0, 0, 1, 0, 0])) >= 1e-10).any()
if condition1 or condition2:
    print("One-hot not working as intended!")
else:
    print("Sanity Check Passed!")

Once you've confirmed the correctness of your solutions with the sanity checks, run the following cell to train our autoencoder on a text corpus derived from wikipedia articles. We will only train our autoencoder on the first $N$ characters of this corpus for some large $N$, since attempting to train on the entire corpus would be very computationally involved.

In [ ]:
# Hyperparameters, feel free to tune these
latent_space_dim = 300       # Dimension of Latent Space
learning_rate = 0.001        # Learning rate for SGD
num_epochs = 3               # Number of SGD epochs
window_length = 3            # Window length for positive examples
num_chars = 1000000          # Number of characters to read from corpus
test_word = 'philosopher'    # Word to use in order to test results

text_file = r"wiki_en.txt"
file = open(text_file, 'r')
corpus = file.read(num_chars)
corpus = re.sub(r'/\'', ' ', corpus)
corpus = re.sub(r'[^\w\s]', '', corpus)
corpus = re.split(r'\s', corpus)
corpus = list(filter(''.__ne__, corpus))
# nltk.download('stopwords')
# READ ME!!! If the next line of code throws an error, uncomment the code above and rerun
from nltk.corpus import stopwords
stops = set(stopwords.words('english'))
corpus = [word for word in corpus if word not in stops]
file.close()
embed = Embedding(latent_space_dim, corpus, window=window_length).fit(learning_rate, num_epochs)
# embed.save()
print(f"Context words for '{test_word}': {embed.most_similar(test_word)[:5]}")

In [ ]:
# Feel free to run further tests on saved weights here


Describe the quality of the results you saw above. Try tuning hyperparameters and retraining if the results are low-quality. Note that the Embedding class allows users to save weights and load pre-trained weights, so once a model is trained in the upper cell, you can instantiate a new embedding with pre-trained weights in the lower cell to experiment further (even if you somehow destroyed the original Embedding object from the upper cell). If you're able to obtain an embedding you believe captures relevant information about the words in the corpus and how they are related, explain why you believe so below. If not, explain why you don't believe the embedding is working well.
##### start answer 1#####

##### end answer 1#####

## Part II: The Embeddings

Now we elaborate upon the embedding-focused approach to Latent Semantic Analysis. In order to derive this approach, we need to name some elements of our autoencoder. For any input vector $\mathbf{x}$, let $\mathbf{\phi}(\mathbf{x})$ be the resulting hidden layer activations. Note that $\mathbf{\phi}(\cdot)$ is a linear transformation from one-hot space into latent space since our hidden layer had no nonlinearity. Similarly, for any hidden layer activation vector $\mathbf{h}$, let $\mathbf{\psi}^T(\mathbf{h})$ be the pre-nonlinearity activations of the final layer; as the notation suggests, $\mathbf{\psi}^T(\cdot)$ is then a linear transformation from latent space to one-hot space, and thus $\mathbf{\psi}(\cdot)$ is a linear transformation from one-hot space to latent space (or more formally, their respective dual spaces, though this is irrelevant to what follows). 

Recall that when we constructed positive examples above, the *center* word was our input word, and we hoped to retrieve relevant *context* words in the outputs. Note also that $\mathbf{\phi}$ embeds inputs, while $\mathbf{\psi}$ embeds outputs. Thus, it seems natural that $\mathbf{\phi}$ is a latent space embedding of center words, while $\mathbf{\psi}$ is a latent space embedding of context words. If a given word $w'$ is a context word for $w$, this means that when we run $w$ under the autoencoder, the component of the output corresponding to $w'$ is large. Let $\mathbf{w}$ and $\mathbf{w'}$ be the one-hot encodings of $w$ and $w'$ respectively. Then the output of the neural network on input $\mathbf{w}$ is 

$$\mathbf{w'}^T\operatorname{sigmoid}\left(\mathbf{\psi}^T\mathbf{\phi}\mathbf{w}\right) = \operatorname{sigmoid}\left(\mathbf{w'}^T\mathbf{\psi}^T\mathbf{\phi}\mathbf{w}\right) = \frac{1}{1 + \exp\left(-\mathbf{w'}^T\mathbf{\psi}^T\mathbf{\phi}\mathbf{w}\right)}$$

Now we digress slightly and shift our focus towards the classification perspective. Recall that from this perspective, the autoencoder is attempting to classify whether $w'$ is a context word for $w$ (for all possible $w'$ at once). We saw that if we run the autoencoder on the one-hot encoding of $w$, the component of the output corresponding to $w'$ represents a classification score for $(w', w)$. It's also strictly between 0 and 1, so we might be tempted to go out on a limb and interpret this as the **probability that $w'$ is a context word for $w$**. We would then have

$$P(w'\text{ is a context word for }w) = \frac{1}{1 + \exp\left(-\mathbf{w'}^T\mathbf{\psi}^T\mathbf{\phi}\mathbf{w}\right)}$$

Suppose we are given a list of tuples $\{(w'_i, w_i, t_i)\}_{i\leq n}$ where $t_i = 1$ if $w'_i$ is a context word for $w_i$ and $t_i = 0$ otherwise. Assuming our model is correct, the probability of these tuples is

$$P(t_0, \dots, t_n\mid w_0, w'_0, \dots, w_n, w'_n) = \prod_{i\leq n}\left(\frac{1}{1 + \exp\left(-\mathbf{w'}_i^T\mathbf{\psi}^T\mathbf{\phi}\mathbf{w}_i\right)}\right)^{t_i}\left(1 - \frac{1}{1 + \exp\left(-\mathbf{w'}_i^T\mathbf{\psi}^T\mathbf{\phi}\mathbf{w}_i\right)}\right)^{1 - t_i}$$

One way to find a correct model is to gather some empirically valid tuples and attempt to maximize their probability under our model. Note, however, that maximizing the function above seems difficult given the intractable product and exponents. To solve this, we can take the logarithm of both sides, and since the logarithm is monotonically increasing, maximizing $\log P$ will be equivalent to maximizing $P$:

$$\log P(t_0, \dots, t_n\mid w_0, w'_0, \dots, w_n, w'_n) = \sum_{i\leq n}\left[t_i\log\left(\frac{1}{1 + \exp\left(-\mathbf{w'}_i^T\mathbf{\psi}^T\mathbf{\phi}\mathbf{w}_i\right)}\right) + (1 - t_i)\log\left(1 - \frac{1}{1 + \exp\left(-\mathbf{w'}_i^T\mathbf{\psi}^T\mathbf{\phi}\mathbf{w}_i\right)}\right)\right]$$

Hmmmmm... this looks familiar. Notice that this is simply the cross-entropy loss of the autoencoder acting on our data! When we originally trained the autoencoder, we had to include every possible pair $(w', w)$ in our data, but this form of the optimization problem gives us a way to include only as many pairs as we need. As discussed before, the large majority of examples drawn from actual text corpuses are negative, leading to class imbalance and slow training. To fix these issues, we can reframe the optimization problem in terms of the embeddings $\mathbf{\phi}(\mathbf{w}_i)$ and $\mathbf{\psi}(\mathbf{w}_i)$ for all words $w$ in the corpus.

First, we simply our expression above:

\begin{align*}\log P(t_0, \dots, t_n\mid w_0, w'_0, \dots, w_n, w'_n) &= \sum_{i\leq n}\left[t_i\log\left(\frac{1}{1 + \exp\left(-\mathbf{w'}_i^T\mathbf{\psi}^T\mathbf{\phi}\mathbf{w}_i\right)}\right) + (1 - t_i)\log\left(\frac{\exp\left(-\mathbf{w'}_i^T\mathbf{\psi}^T\mathbf{\phi}\mathbf{w}_i\right)}{1 + \exp\left(-\mathbf{w'}_i^T\mathbf{\psi}^T\mathbf{\phi}\mathbf{w}_i\right)}\right)\right]\\&=\sum_{i\leq n}\left[(1 - t_i)\log\exp\left(-\mathbf{\psi}(\mathbf{w'}_i)^T\mathbf{\phi}(\mathbf{w}_i)\right) - (1 - t_i + t_i)\log \left(1 + \exp\left(-\mathbf{w'}_i^T\mathbf{\psi}^T\mathbf{\phi}\mathbf{w}_i\right)\right)\right]\\&=-\sum_{i\leq n}\left[(1 - t_i)\mathbf{\psi}(\mathbf{w'}_i)^T\mathbf{\phi}(\mathbf{w}_i) + \log \left(1 + \exp\left(-\mathbf{w'}_i^T\mathbf{\psi}^T\mathbf{\phi}\mathbf{w}_i\right)\right)\right]\end{align*}

With this expression in a simpler form, we can evaluate its gradient with respect to the embedding vectors:

\begin{align*}\nabla_{\mathbf\phi(\mathbf w_i)}\log P &= -\left[(1 - t_0)\mathbf\psi(\mathbf w'_i) - \frac{e^{-\mathbf\psi(\mathbf w'_i)^T\mathbf\phi(\mathbf w_i)}}{1 + e^{-\mathbf\psi(\mathbf w'_i)^T\mathbf\phi(\mathbf w_i)}}\mathbf\psi(\mathbf w'_i)\right]\\&=\left[t_0 - \frac{1}{1 + e^{-\mathbf\psi(\mathbf w'_i)^T\mathbf\phi(\mathbf w_i)}} \right] \mathbf\psi(\mathbf w'_i)\\\nabla_{\mathbf\psi(w_i')}\log P &= -\left[(1 - t_0)\mathbf\phi(\mathbf w'_i) - \frac{e^{-\mathbf\psi(\mathbf w'_i)^T\mathbf\phi(\mathbf w_i)}}{1 + e^{-\mathbf\psi(\mathbf w'_i)^T\mathbf\phi(\mathbf w_i)}}\mathbf\phi(\mathbf w_i)\right]\\&=\left[t_0 - \frac{1}{1 + e^{-\mathbf\psi(\mathbf w'_i)^T\mathbf\phi(\mathbf w_i)}} \right] \mathbf\phi(\mathbf w_i)\end{align*}

Since we hope to maximize $\log P$, we can produce the embeddings with gradient ascent! In this format, we also have the power to choose the examples $(w', w)$ on which we train, meaning we can avoid training on too many negative examples, and ultimately we can avoid the slow training times and class imbalance which the autoencoder suffers from. Now let's implement an embedding class which performs gradient ascent as described above.

In [ ]:
class AltEmbedding:

    def __init__(self, latent_dim, corpus, window=3):
        
        # Separate unique words and find one-hot encoding indices
        self.words = set(corpus)
        ################## BEGIN PART (I) ##################
        # Create a one-hot encoding of the set of words
        # HINT: how did we do this last time?

        ################### END PART (I) ###################
        self.freqs = Counter([idx[word] for word in corpus])
        self.idx = {i: idx[word] for i, word in enumerate(corpus)}
        print(f"found words {len(self.words)} words in a corpus of length {len(corpus)}")
        
        # Initialize weights
        self.center = np.random.randn(len(self.words), latent_dim)
        self.context = np.random.randn(latent_dim, len(self.words))
        self.window = window

    def fit(self, lr, epochs):
        for i in range(epochs):
            data_bar = tqdm(range(len(self.words)), position=0, leave=True)
            data_bar.set_description(f"Processing epoch {i+1} out of {epochs}")
            for j in data_bar:
                center_index = self.idx[j]
                ################## BEGIN PART (J) ##################
                # Find the start and end indices of the window
                # HINT: Remember to ensure the start end end indices are
                #       not out of range!

                ################### END PART (J) ###################
                for k in range(start, end):
                    if k == 0: next
                    context_index = self.idx[k]
                    e = np.exp(-(self.center[[center_index], :] @ self.context[:, [context_index]])[0][0])
                    ################## BEGIN PART (K) ##################
                    # Perform Gradient Ascent updates for positive examples
                    # HINT: We have already calcualted th exponential term for you.
                    # HINT: See the gradient ascent updates for negative examples if
                    #       you have trouble. Remember to adjust the coefficient based
                    #       on the value of t_i (these are positive examples, not negative)

                    ################### END PART (K) ###################
                neg_index = choices(range(len(self.words)), weights=self.freqs, k=2*self.window)
                for k in neg_index:
                    context_index = k
                    e = np.exp(-(self.center[[center_index], :] @ self.context[:, [context_index]])[0][0])
                    self.center[[center_index], :] += lr * -1 / (1 + e) * self.context[:, [context_index]].T
                    self.context[:, [context_index]] += lr * -1 / (1 + e) * self.center[[center_index], :].T
                    
    def save(self):
        kwargs = {'protocol': pickle.HIGHEST_PROTOCOL}
        np.save("center", self.center)
        np.save("context", self.context)
        with open('alt_word_indices.pickle', 'wb') as handle:
            pickle.dump(self.idx, handle, **kwargs)

    def load(self):
        self.center = np.load("center")
        self.context = np.load("context")
        with open('alt_word_indices.pickle', 'rb') as handle:
            self.idx = pickle.load(handle)

Once you complete the code above, run the following cell to try out your embeddings.

In [ ]:
# Hyperparameters, feel free to tune these
latent_space_dim = 300       # Dimension of Latent Space
learning_rate = 0.001        # Learning rate for SGD
num_epochs = 3               # Number of SGD epochs
window_length = 3            # Window length for positive examples
num_chars = 1000000          # Number of characters to read from corpus
test_word = 'philosopher'    # Word to use in order to test results

embed = AltEmbedding(latent_space_dim, corpus, window=window_length).fit(learning_rate, num_epochs)
# embed.save()
print(f"Context words for '{test_word}': {embed.most_similar(test_word)[:5]}")

In [ ]:
# Feel free to run further tests on saved weights here


Describe the quality of the results you saw above. Try tuning hyperparameters and retraining if the results are low-quality. Note that the Embedding class allows users to save weights and load pre-trained weights, so once a model is trained in the upper cell, you can instantiate a new embedding with pre-trained weights in the lower cell to experiment further (even if you somehow destroyed the original Embedding object from the upper cell). If you're able to obtain an embedding you believe captures relevant information about the words in the corpus and how they are related, explain why you believe so below. If not, explain why you don't believe the embedding is working well.
##### start answer 2#####

##### end answer 2#####

## Part 3: The Applications
It turns out that the gensim package handles a lot of heavy lifting for us. Now that we have a good idea of what is going on under the hood of word2vec and Latent Semantic Analysis, we will see some implementations of such algorithms.

a) First, we are going to see how word2vec can be used to find word similarities using gensim. The download_data_as_text method will return a large text block of reuters articles with ~1,000,000 words.


In [5]:
def download_data_as_text():
    all_articles = ""
    for article in reuters.fileids():
        all_articles += reuters.raw(article)
    
    filename=open("reuters.txt", "w")
    filename.write(all_articles)
    return all_articles

# Generate the model using Word2Vec and save it in a file called reuters.word2vec
def train_word2vec(text):
    # Begin part 1
    
    
    # End part 1

# Load the model in reuters.word2vec to create a function that takes in a word and outputs the 10 most similar words
# semantically to the given word.
def word_sim(word):
    # Begin part 2
    
    
    
    
    # End part 2

text = download_data_as_text()
train_word2vec()
word_sim("economy")
word_sim("corn")

IndentationError: expected an indented block (<ipython-input-5-33448df78dab>, line 15)

b) Now that we have seen word2vec in action we can take a look at the built-in latent semantic analysis model for gensim. We were not as concerned with data preprocessing for the previous model given that we just wanted word similarities. However, now we will concern ourselves with processing and tokenizing data as without it, LSA will not word very well. A token in NLP is the fundamental building block on which all training is done. A token is a string of contiguous characters between two spaces, punctuations marks, or any combination there of. An example of a token is a word, but it also includes numbers and abbreviations. A stop word is a set of the most commonly used words in the english dictionary. While on face it might seem like we should include these, the most common words are "the", "be", "to", "of", and other assorted prepositions and articles. Clearly these do not represent any underlying 'topic' so we shoudl removoe them. A stemmed word is one where extraneous elements such as plurals or tense changes are removed. While the those do somewhat affect the underlying topic, they do affect the math in generating said topics, so we should remove them.

In [ ]:
def download_as_tokens():
    all_articles=[]
    for article in gutenberg.fileids():
        all_articles.append(gutenberg.raw(article))
    tokens = []
    tokenizer = RegexpTokenizer(r'\w+')
    stopping_words = set(stopwords.words('english'))
    stemmer = PorterStemmer()
    for article in all_articles:
        # Begin part b
        
        
        
        
        # End part b
    print("finished creating tokens")
    return tokens

c) Now we want to actually train our LSA model. Note that gensim calls their function LSIModel (standing for Latent Semantic Indexing which is just another phrase for the same idea). You can return the model, save it in a file called gutenberg.LSIModel, or both. We can fix the number of topics we want to generate for now to be 100.

In [ ]:
def train_LSIModel(tokens, num_top):
    # Begin part c
    
    
    
    # End part c

d) The last thing we want to do is print out what the top categories consist of in our trained model. Use the print_topics method built into an LSIModel on gensim.

In [ ]:
def doc_sim():
    # Begin part d
    
    # End part d

(Optional) e) Gensim also has built-in code that contains a CoherenceModel. We can think about the coherence as a way to evaluate the effectiveness of our model in seperating our topics into distinct, non-overlapping categories. What we want to do is iterate over some range that represents a reasonable number of topics our corpus could conceivably contain. Then we will find the coherence of each model trained on a certain number of topics and choose the maximum of them. That will be the final model we use. (One thing to note is that CoherenceModel method has had some problems on Windows which is why we made this optional, don't worry if you can't entirely get this to work).

In [4]:
def find_best_coherence(tokens, range_num_topics):
    # Begin part e
    
    
    
    
    
    
    
    
    
    
    
    # End part e

SyntaxError: unexpected EOF while parsing (<ipython-input-4-e06bbe59efce>, line 14)

We recommend that you try this out with whatever corpus you want. nltk has many free corpuses that are availabe to import. Additionally if you have texts of your own that you want to classify, you can load it into the download_as_tokens method and proceed from there. 

Write below things that you notice about the top categories. Play around with the number of words being outputted to see the full scope of what a category means. Some questions to think about as you are observing the outputs:
- What does a negative value mean in a category?
- Are the topics necessarily easy to define in human terms?

##### Answer question here: #####


We hope you enjoyed this notebook!